In [1]:
# Importação de bibliotecas
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

# Parâmetros de conexão
host = 'localhost'
port = 5432
database = 'transporte_coletivo'
user = 'postgres'
password = '123456789'

In [2]:
# Função para abrir conexão com o BD
def abre_conexao():
    
    conexao = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conexao.cursor()
    
    return conexao, cursor

In [3]:
# Função para fechar conexão com o BD
def fecha_conexao(conexao, cursor):
    
    cursor.close()
    conexao.close()

In [4]:
# Função para criar esquema
def cria_esquema():
    
    conexao, cursor = abre_conexao()
    cursor.execute(f"CREATE SCHEMA RODOVIARIO;")
    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    return print("Esquema criado com sucesso!")
#cria_esquema('rodoviario')

In [5]:
def exclui_esquema():
    
    conexao, cursor = abre_conexao()
    cursor.execute(f"DROP SCHEMA IF EXISTS RODOVIARIO CASCADE;")
    conexao.commit()
    cursor.close()
    fecha_conexao(conexao, cursor)

    return print("Esquema excluído com sucesso!")

#exclui_esquema('rodoviario')

In [6]:
# Função para criar a estrutura de tabelas
def cria_todas_tabelas():

    conexao, cursor = abre_conexao()

    diretorio = r'/home/etopocart/Documentos/Trabalho Final/BD/cria_banco.sql'
    with open(diretorio, 'r') as arquivo:
        consultas = arquivo.read().split(';')

    for consulta in consultas:
        if consulta.strip():
            cursor.execute(consulta)

    conexao.commit()
    fecha_conexao(conexao, cursor)

    return print("Tabelas criadas com sucesso!")

In [7]:
def popula_todas_tabelas():

    conexao, cursor = abre_conexao()

    diretorio = r'/home/etopocart/Documentos/Trabalho Final/BD/popula_banco.sql'
    with open(diretorio, 'r') as arquivo:
        consultas = arquivo.read().split(';')

    for consulta in consultas:
        if consulta.strip():
            cursor.execute(consulta)

    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    return print("Tabelas populadas com sucesso!")

In [8]:
def recupera_tabelas():
    
    conexao, cursor = abre_conexao()

    cursor.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema='rodoviario';")
    tabelas = cursor.fetchall()
   
    nomes_tabelas = [tabela[0] for tabela in tabelas]
    
    return nomes_tabelas

In [9]:
def exclui_todas_tabelas():
     
    conexao, cursor = abre_conexao()

    nomes_tabelas = recupera_tabelas()
    
    for nome_tabela in nomes_tabelas:
        cursor.execute(f"DROP TABLE IF EXISTS rodoviario.{nome_tabela} CASCADE;")
        #print(nome_tabela)

    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    return print("Tabelas excluídas com sucesso!")

In [10]:
def deleta_bd_completo ():
    
    exclui_todas_tabelas()
    exclui_esquema()

In [11]:
def cria_bd_completo ():
    
    cria_esquema()
    cria_todas_tabelas()
    popula_todas_tabelas()

In [12]:
def insere_registro():

    conexao, cursor = abre_conexao()

    tabelas = recupera_tabelas()
    
    #Alguns inserts estão programados para demonstrar falhas de integridade referencial
    comandos = [
        ("INSERT INTO rodoviario.MUNICIPIO (id_municipio, nome, tarifa) VALUES (6, 'Cocal do Sul', 3.75)"),
        ("INSERT INTO rodoviario.TERMINAL (id_terminal, nome, FK_MUNICIPIO_id_municipio) VALUES (11, 'Terminal K', 3)"),
        ("INSERT INTO rodoviario.VEICULO (id_veiculo, placa, ano, FK_ITINERARIO_id_itinerario, FK_EMPRESA_id_empresa, FK_TIPO_ONIBUS_id_tipo_onibus) VALUES (6, 'AAA1234', 2021, 3, 3, 2)"),
        ("INSERT INTO rodoviario.MOTORISTA (id_motorista, nome, chn, validade_cnh, FK_VEICULO_id_veiculo) VALUES (6, 'Motorista F', '86420975318', '2023-08-15', 1)"),
        ("INSERT INTO rodoviario.ITINERARIO (id_itinerario, tipo) VALUES (4, 'Manhã')"),
        ("INSERT INTO rodoviario.EMPRESA (id_empresa, nome, cnpj) VALUES (6, 'Empresa F', 654321987)"),
        ("INSERT INTO rodoviario.TIPO_ONIBUS (id_tipo_onibus, tipo) VALUES (4, 'Double decker')"),
        ("INSERT INTO rodoviario.LINHA (id_linha, nome) VALUES (6, 'Linha 6')"),
        ("INSERT INTO rodoviario.LINHA_PARADA (id_linha_parada, FK_LINHA_id_linha, FK_PARADA_id_parada) VALUES (7, 2, 1)"),
        ("INSERT INTO rodoviario.PARADA (id_parada, endereco_rua, endereco_numero) VALUES (6, 'Rua F', 600)"),
        ("INSERT INTO rodoviario.MEIO_PAGAMENTO (id_meio_pagamento, meio_pagamento, bandeira_cc, tipo_vt) VALUES (6, 'Pix', NULL, NULL)"),
        ("INSERT INTO rodoviario.MEIO_PAGAMENTO_TERMINAL (id_meio_pagamento_terminal, FK_MEIO_PAGAMENTO_id_meio_pagamento, FK_TERMINAL_id_terminal) VALUES (11, 1, 11)"),
        ("INSERT INTO rodoviario.CONEXAO (id_conexao, FK_LINHA_id_linha, FK_TERMINAL_id_terminal) VALUES (11, 1, 11)"),
        ("INSERT INTO rodoviario.LINHA_PARADA_ITINERARIO (id_linha_parada_itinerario, FK_LINHA_PARADA_LINHA_PARADA_id_linha_parada, FK_ITINERARIO_id_itinerario) VALUES (11, 2, 2)")
        ]
    
    print("Em qual tabela deseja inserir um registro?")

    menu = {}
    for i, valor in enumerate(tabelas):
        menu[i] = valor
        print(f"{i}: {valor}")
    
    escolha = int(input('Digite o identificador:'))
    while escolha not in menu:
        print("Escolha inválida. Tente novamente.")
        escolha = int(input("Digite o identificador:"))
    
    print(f"\nVocê escolheu a tabela {menu[escolha]}")

    if escolha >= 0 and escolha < len(comandos):
        comando_sql = comandos[escolha]
        try:
            cursor.execute(comando_sql)
            print("\nInserção realizada com sucesso!")
        except psycopg2.errors.ForeignKeyViolation:
            print("Não é possível inserir o registro devido a restrições de integridade referencial.")
    
    conexao.commit()
    fecha_conexao(conexao, cursor)  

In [13]:
def apaga_registro():
    
    conexao, cursor = abre_conexao()

    tabelas = recupera_tabelas()

    print("De qual tabela deseja apagar um registro?")

    menu = {}
    for i, valor in enumerate(tabelas):
        menu[i] = valor
        print(f"{i}: {valor}")
    
    escolha = int(input('Digite o identificador:'))
    while escolha not in menu:
        print("Escolha inválida. Tente novamente.")
        escolha = int(input("Digite o identificador:"))
    
    print(f"\nVocê escolheu a tabela {menu[escolha]}")
    
    cursor.execute(f"SELECT * FROM rodoviario.{menu[escolha]}")
    dados = cursor.fetchall()
    
    print(f"Esses são os dados: \n{dados}")
    
    registro_id = int(input("Digite o ID do registro que deseja excluir: "))

    if registro_id in [linha[0] for linha in dados]:
        comando_sql = f"DELETE FROM rodoviario.{menu[escolha]} WHERE id_{menu[escolha]} = {registro_id}"
        try:
            cursor.execute(comando_sql)
            print("Registro excluído com sucesso!")
        except psycopg2.errors.ForeignKeyViolation:
            print("Não é possível excluir o registro devido a restrições de integridade referencial.")
    
    conexao.commit()
    fecha_conexao(conexao, cursor) 
    
#     conexao.commit()
#     fecha_conexao(conexao, cursor)

In [14]:
def altera_registro():
    
    conexao, cursor = abre_conexao()

    tabelas = recupera_tabelas()
    
    #Alguns updates estão pensados para demonstrar falhas de integridade referencial
    comandos = [
        ("UPDATE rodoviario.MUNICIPIO SET tarifa = 5.0 WHERE id_municipio = 2"),
        ("UPDATE rodoviario.TERMINAL SET nome = 'Terminal X' WHERE id_terminal = 5"),
        ("UPDATE rodoviario.VEICULO SET ano = 2022 WHERE id_veiculo = 3"),
        ("UPDATE rodoviario.MOTORISTA SET nome = 'Motorista X' WHERE id_motorista = 4"),
        ("UPDATE rodoviario.ITINERARIO SET tipo = 'Tarde' WHERE id_itinerario = 2"),
        ("UPDATE rodoviario.EMPRESA SET nome = 'Empresa Z' WHERE id_empresa = 1"),
        ("UPDATE rodoviario.TIPO_ONIBUS SET tipo = 'Simples' WHERE id_tipo_onibus = 1"),
        ("UPDATE rodoviario.LINHA SET nome = 'Linha Y' WHERE id_linha = 3"),
        ("UPDATE rodoviario.LINHA_PARADA SET FK_LINHA_id_linha = 3, FK_PARADA_id_parada = 2 WHERE id_linha_parada = 2"),
        ("UPDATE rodoviario.PARADA SET endereco_rua = 'Rua Z', endereco_numero = 700 WHERE id_parada = 4"),
        ("UPDATE rodoviario.MEIO_PAGAMENTO SET meio_pagamento = 'Cartão de Crédito', bandeira_cc = 'Visa' WHERE id_meio_pagamento = 3"),
        ("UPDATE rodoviario.MEIO_PAGAMENTO_TERMINAL SET FK_MEIO_PAGAMENTO_id_meio_pagamento = 10 WHERE id_meio_pagamento_terminal = 1"),
        ("UPDATE rodoviario.CONEXAO SET FK_LINHA_id_linha = 2, FK_TERMINAL_id_terminal = 3 WHERE id_conexao = 6"),
        ("UPDATE rodoviario.LINHA_PARADA_ITINERARIO SET FK_LINHA_PARADA_LINHA_PARADA_id_linha_parada = 13 WHERE id_linha_parada_itinerario = 7")
]
    
    print("De qual tabela deseja alterar um registro?")

    menu = {}
    for i, valor in enumerate(tabelas):
        menu[i] = valor
        print(f"{i}: {valor}")
    
    escolha = int(input('Digite o identificador:'))
    while escolha not in menu:
        print("Escolha inválida. Tente novamente.")
        escolha = int(input("Digite o identificador:"))
    
    print(f"\nVocê escolheu a tabela {menu[escolha]}")
    
    cursor.execute(f"SELECT * FROM rodoviario.{menu[escolha]}")
    dados = cursor.fetchall()
    
    print(f"Esses são os dados: \n{dados}")
    
    #registro_id = int(input("Digite o ID do registro que deseja alterar: ")) 

    if escolha >= 0 and escolha < len(comandos):
        comando_sql = comandos[escolha]
        try:
            cursor.execute(comando_sql)
            print("Registro alterado com sucesso!")
        except psycopg2.errors.ForeignKeyViolation:
            print("Não é possível alterar o registro devido a restrições de integridade referencial.")
    
    
    cursor.execute(f"SELECT * FROM rodoviario.{menu[escolha]}")
    dados = cursor.fetchall()
    
    print(f"Esses são os dados alterados: \n{dados}")
    
    conexao.commit()
    fecha_conexao(conexao, cursor)

In [15]:
def sql1():
     
    conexao, cursor = abre_conexao()
    
    cursor.execute(f"SELECT linha.nome, COUNT(parada) AS total_paradas \
                    FROM rodoviario.linha \
                    inner join rodoviario.linha_parada on linha.id_linha = linha_parada.fk_linha_id_linha \
                    inner join rodoviario.parada on linha_parada.fk_parada_id_parada = parada.id_parada \
                    group by linha.nome")
    resultado = cursor.fetchall()
    
    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    
    def gera_grafico(resultado):

        df = pd.DataFrame(resultado, columns=['Linha', 'Total de Paradas'])

        df = df.sort_values('Total de Paradas', ascending=False)

        plt.bar(df['Linha'], df['Total de Paradas'])
        plt.xlabel('Linha')
        plt.ylabel('Total de Paradas')
        plt.title('Total de Paradas por Linha')
        plt.xticks(rotation=45)  
        plt.tight_layout()
        plt.show()

    grafico = gera_grafico(resultado)
    
    return grafico

In [16]:
def sql2():
     
    conexao, cursor = abre_conexao()
    
    cursor.execute(f"SELECT municipio.nome, COUNT(itinerario) AS total_itinerarios_noturnos \
                    FROM rodoviario.itinerario \
                    INNER JOIN rodoviario.linha_parada_itinerario ON itinerario.id_itinerario = linha_parada_itinerario.FK_ITINERARIO_id_itinerario \
                    INNER JOIN rodoviario.linha_parada ON linha_parada_itinerario.FK_LINHA_PARADA_LINHA_PARADA_id_linha_parada = linha_parada.id_linha_parada \
                    INNER JOIN rodoviario.linha ON linha_parada.FK_LINHA_id_linha = linha.id_linha \
                    INNER JOIN rodoviario.conexao ON linha.id_linha = conexao.FK_LINHA_id_linha \
                    INNER JOIN rodoviario.terminal ON conexao.FK_TERMINAL_id_terminal = terminal.id_terminal \
                    INNER JOIN rodoviario.municipio ON terminal.FK_MUNICIPIO_id_municipio = municipio.id_municipio \
                    WHERE itinerario.tipo = 'Noturno' \
                    GROUP BY municipio.nome;")
    resultado = cursor.fetchall()
    
    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    
    def gera_grafico(resultado):
        df = pd.DataFrame(resultado, columns=['Município', 'Total de Itinerários Noturnos'])

        df = df.sort_values('Total de Itinerários Noturnos', ascending=False)

        plt.bar(df['Município'], df['Total de Itinerários Noturnos'])
        plt.xlabel('Município')
        plt.ylabel('Total de Itinerários Noturnos')
        plt.title('Total de Itinerários Noturnos por Município')
        plt.xticks(rotation=45)  
        plt.tight_layout() 
        plt.show()
        
    grafico = gera_grafico(resultado)
    
    return grafico

In [17]:
def sql3():
     
    conexao, cursor = abre_conexao()
    
    cursor.execute(f" SELECT  i.tipo, avg(ano) \
                    FROM rodoviario.LINHA l \
                    INNER JOIN rodoviario.LINHA_PARADA lp ON lp.FK_LINHA_id_linha = l.id_linha \
                    INNER JOIN rodoviario.linha_parada_itinerario lpi ON lp.id_linha_parada = lpi.fk_linha_parada_linha_parada_id_linha_parada \
                    INNER JOIN rodoviario.ITINERARIO i ON i.id_itinerario = lpi.fk_itinerario_id_itinerario \
                    INNER JOIN rodoviario.VEICULO v ON v.FK_ITINERARIO_id_itinerario = i.id_itinerario\
                    GROUP BY i.tipo")
                    
    resultado = cursor.fetchall()
    
    conexao.commit()
    fecha_conexao(conexao, cursor)
    
    
    def gera_grafico(resultado):

        df = pd.DataFrame(resultado, columns=['Tipo', 'Média de Ano'])

        plt.figure(figsize=(10, 6))

        tipos = df['Tipo']
        media_ano = df['Média de Ano']
        
        cores = ['red', 'green', 'blue']

        plt.bar(tipos, media_ano, color=cores)

        plt.title('Média de Ano de Fabricação por Tipo de Itinerário')

        plt.ylim(1980, max(media_ano) + 100)

        plt.xticks(rotation=45)

        plt.show()
        
    grafico = gera_grafico(resultado)
    
    return grafico

In [18]:
def exibir_menu():
    print("===== MENU =====")
    print("1. Criar banco de dados completo")
    print("2. Deletar banco de dados completo")
    print("3. Inserir registro")
    print("4. Apagar registro")
    print("5. Alterar registro")
    print("6. Executar consulta SQL1")
    print("7. Executar consulta SQL2")
    print("8. Executar consulta SQL3")
    print("9. Sair")
    print("================\n")

def executar_menu():
    while True:
        exibir_menu()
        opcao = input("Digite o número da opção desejada: ")
        print()
        
        if opcao == "1":
            cria_bd_completo ()
        elif opcao == "2":
            deleta_bd_completo()
        elif opcao == "3":
            insere_registro()
        elif opcao == "4":
            apaga_registro()
        elif opcao == "5":
            altera_registro()
        elif opcao == "6":
            sql1()
        elif opcao == "7":
            sql2()
        elif opcao == "8":
            sql3()
        elif opcao == "9":
            print("Encerrando o programa.")
            break
        else:
            print("Opção inválida. Por favor, escolha uma opção válida.\n")

In [ ]:
executar_menu()

===== MENU =====
1. Criar banco de dados completo
2. Deletar banco de dados completo
3. Inserir registro
4. Apagar registro
5. Alterar registro
6. Executar consulta SQL1
7. Executar consulta SQL2
8. Executar consulta SQL3
9. Sair

Digite o número da opção desejada: 4

De qual tabela deseja apagar um registro?
0: municipio
1: terminal
2: veiculo
3: motorista
4: itinerario
5: empresa
6: tipo_onibus
7: linha
8: linha_parada
9: parada
10: meio_pagamento
11: meio_pagamento_terminal
12: conexao
13: linha_parada_itinerario
Digite o identificador:12

Você escolheu a tabela conexao
Esses são os dados: 
[(1, 1, 2), (2, 2, 3), (3, 3, 4), (4, 4, 5), (5, 5, 6), (7, 2, 8), (8, 3, 9), (9, 4, 10), (10, 5, 1), (6, 2, 3)]
Digite o ID do registro que deseja excluir: 1
Registro excluído com sucesso!
===== MENU =====
1. Criar banco de dados completo
2. Deletar banco de dados completo
3. Inserir registro
4. Apagar registro
5. Alterar registro
6. Executar consulta SQL1
7. Executar consulta SQL2
8. Executar c